In [3]:
#initialize
#     model
#     environment variables

import sys
sys.path.append('..')
from src.utils.llamaindex_retriever import LlamaIndexRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
from langchain.chat_models import AzureChatOpenAI


def load_env_variables(file_path):
    load_dotenv(file_path)
    print("Environment variables loaded successfully!")

env_file_path = "../.env"
load_env_variables = load_env_variables(env_file_path)
max_tokens = 3500
temperature = 0.1

# embeddings = AzureOpenAIEmbeddings(azure_deployment=azure_deployment, openai_api_version=openai_api_version)
embeddings =  AzureOpenAIEmbeddings(
        deployment=os.getenv("EMB_DEPLOYMENT"),
        openai_api_version=os.getenv("EMB_OPENAI_API_VERSION"),
        model=os.getenv("EMB_MODEL"),
        openai_api_key=os.getenv("EMB_OPENAI_API_KEY"),
        openai_api_base=os.getenv("EMB_OPENAI_ENDPOINT"),
        openai_api_type=os.getenv("EMB_API_TYPE"),
    )

llm_service = AzureChatOpenAI(deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'), openai_api_version=os.getenv("OPENAI_API_VERSION"),
                        openai_api_base=os.getenv("OPENAI_API_BASE"), 
                        openai_api_type= os.getenv("OPENAI_API_TYPE"),
                        openai_api_key=os.getenv("OPENAI_API_KEY"),
                        max_tokens=max_tokens,
                        temperature=temperature)

Environment variables loaded successfully!


/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py:113: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://polarisopenai.openai.azure.com/ to https://polarisopenai.openai.azure.com//openai.
  warnings.warn(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py:1

In [4]:
#define utils
from src.utils.cube_semantic_custom import CubeSemanticLoader
def fetch_cube_metadata(*args, **kwargs):
    try:
        # # Load document from Cube meta api
        loader = CubeSemanticLoader(os.getenv("CUBE_API_URL"), os.getenv("CUBE_TOKEN"), False)
        documents = loader.load()
        # to_json()
        return documents
    except Exception as e:
        # Handle exceptions gracefully and return an error response
        print("Error in fetching metadata from cube: " + str(e))
        return 0

def create_vector_store(documents, local_vector_store_path, *args, **kwargs):
    print("Loaded documents: " + str(documents))
    vectorstore = FAISS.from_documents(documents, embeddings)
    vectorstore.save_local(local_vector_store_path)
    print("Vector store created and saved successfully!")

def load_vector_store(vector_store_path, embeddings, *args, **kwargs):
    # Load the vector store from the local file system
    vectorstore = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
    print("Vector store loaded successfully!")
    
    return vectorstore

In [5]:
#load existing vector store
vector_store_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vector_store/cube_meta_faiss_index"
vectorstore = load_vector_store(vector_store_path, embeddings)

Vector store loaded successfully!


Tools 

In [6]:
import json
from crewai import Agent, Task, Crew
from langchain.tools import tool
from src.utils.llamaindex_retriever import LlamaIndexRetriever
from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
# from langchain.tools import BaseTool, StructuredTool, tool
from crewai_tools import BaseTool


def get_similar_documents_faiss(query, max_number_documents=3):
  vectorstore = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
  docs = vectorstore.similarity_search_with_relevance_scores(query, max_number_documents)
  relevant_documents = []
  for doc in docs:
      doc = doc[0]
      meta = {'text':doc.page_content, 'table_metadata': doc.metadata}
      relevant_documents.append(meta)
  return relevant_documents

def get_similar_documents(query, max_number_documents=3):
    return get_similar_documents_faiss(query, max_number_documents)


class QueryInput(BaseModel):
    query: str = Field(description="should be enquiry query")

class TerminationHandler(CallbackManagerForToolRun):
    pass

class RephraseInputQuery(BaseTool):
    name:str = "rephrase_input_query"
    description :str = "Useful to rephrase the query to capture the intent of the user regarding metric information"
    args_schema: Type[BaseModel] = QueryInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        metric_description = self.rephrase_input_query(query)
        return metric_description

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")
    
    
    def rephrase_input_query(self, query, *args, **kwargs):
        agent = Agent(
                role='Intent Capturer',
                goal=
                'Rephrasing the query to capture the intent of the user regarding metric information',
                backstory=
                "You are an expert to understand the user's intent and rephrase the query to capture the intent of the user accurately.",
                llm = llm_service,
                allow_delegation=False)
        task = Task(
                agent=agent,
                description=
                f'Rephrase the query to capture the intent of the user regarding metric information. The query is {query}. Donot add any noise to the response',
                expected_output="some string",
        
            )
        extracted_metrics = task.execute()

        return extracted_metrics
    

    
class MetricDiscovery(BaseTool):
    name :str = "metric_discovery"
    description:str = """Useful for general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."""
    args_schema: Type[BaseModel] = QueryInput

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        metric_description = self.metric_discovery(query)
        return metric_description

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")
    
    
    def metric_discovery(self, query, *args, **kwargs):
        """Useful for general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."""
        relevant_documents = get_similar_documents(query)
        agent = Agent(
                role='Data Analyst Assistant',
                goal=
                'Empower users to understand and utilize data effectively. This includes helping them discover relevant metrics, interpreting their meaning',
                backstory=
                "The primary purpose is to bridge the gap between raw data and user comprehension, fostering a data-driven culture within the organization.",
                llm = llm_service,
                allow_delegation=False)  
        
        task = Task(
                agent=agent,
                description=
                """ You are responding to  question {metric_description} with answer from the Metadata provided to you as {relevant_documents}}.
                    Strictly answer the question with the information present in metadata only.
                    Respond with "Sorry, the query is out of scope." if the answer is not present in metadata and terminate further reasoning and prcoess.
                    """,
                expected_output="some string",
        
            )
        output = task.execute()

        return output




Tasks

def callback_function(output: TaskOutput):
    # Do something after the task is completed
    # Example: Send an email to the manager
    print(f"""
        Task completed!
        Task: {output.description}
        Output: {output.raw_output}
    """)

In [7]:
# def check_termination(task_output):
#     agent = Agent(
#                 role='Determine query completion',
#                 goal=
#                 'Given task output, determine termination of crew workflow',
#                 backstory=
#                 "Given user query request from user determine if no more new task is needed to be executed",
#                 llm = llm_service,
#                 allow_delegation=False)
#     task = Task(
#                 agent=agent,
#                 description=
#                 f'Given task output, if the query is completed and the task result determines that the query is out of scope then terminate the workflow or else the required query is satisfied. Following is the task result {task_output}',
#                 expected_output="some string",
        
#             )
#     termination_status = task.execute()
#     return termination_status
class MetricDiscoveryTasks():
  def metric_isolation(self, agent, query):
    return Task(description=(f"""
        Rephrase the query to capture the intent of the user regarding metric information. The query is {query}. Donot add any noise to the response.
        {self.__tip_section()}"""),
      agent=agent,
      expected_output="Reformatted query to capture the intent of the user regarding metric information.",
      # callback=callback_function,
    )
  
  def metric_discovery(self, agent):
    return Task(description=(f"""
        Answer the general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns.
        {self.__tip_section()}"""),
      agent=agent,
      expected_output="If relevant metric exists in metadata, provide the answer. Else, respond with 'Sorry, the query is out of scope.' and terminate process.",
      # callback=callback_function,
    )
  def __tip_section(self):
    return "If you do your BEST WORK, I'll give you a $10,000 commission!"
  
  

Agents

In [8]:
class MetricDiscoveryAgent():
  def user_intent_capture(self):
    return Agent(
      role='Intent Capturer',
      goal=
      'Rephrasing the query to capture the intent of the user regarding metric information',
      backstory=
      "You are an expert to understand the user's intent and rephrase the query to capture the intent of the user accurately.",
      verbose=True,
      tools=[
        RephraseInputQuery()
      ],
      llm = llm_service,
    )
  def discover_metric_info(self):
    return Agent(
     role='Data Analyst Assistant',
      goal=
      'Empower users to understand and utilize data effectively. This includes helping them discover relevant metrics, interpreting their meaning',
      backstory=
      "The primary purpose is to bridge the gap between raw data and user comprehension, fostering a data-driven culture within the organization.",
      llm = llm_service,
      verbose=True,
      tools=[
        MetricDiscovery()
      ],
    )

Crew

In [9]:
def after_task_callback(output):
  # Perform actions after the task, 
  # for example, logging or updating agent state
  
  print(f"Agent completed task with result: {output}")

# Assigning the function to task_callback
task_callback = after_task_callback

In [10]:
def test_callback():
        # Accessing local variables
        print("Local variables:", locals())
        
        # Accessing global variables
        print("Global variables:", globals())    
        
        # Accessing built-in variables
        print("Built-in variables:", dir(__builtins__))
        after_task_callback("output")
  

In [11]:
from abc import ABC, abstractmethod
class Expert(ABC):
    name: str = ""
    description: str = ""
    public_description: str = ""
    arg_description: str = "The argument to the function."

    @abstractmethod
    def call(
        self,
        goal: str,
        task: str,
        input_str: str,
    ) -> str:
        pass

In [23]:
from crewai.process import Process
from typing import Any
class MetricDiscoveryInputCrew(Expert):
  name = "MetricDiscovery"
  description = (
       "Use this to answer the general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."
    )
  public_description = "Metric discovery crew"
  arg_description = "Query related to metrics."

  def crew_process(self, query: str):
    agents = MetricDiscoveryAgent()
    tasks = MetricDiscoveryTasks()
    # print(agents)
    # print(tasks)
    user_intent_capture_agent = agents.user_intent_capture()
    discover_metric_info_agent = agents.discover_metric_info()
    # print(metric_isolator_agent)
    # metric_isolator_task = tasks.metric_isolation(metric_isolator_agent, self.query)
    metric_isolator_task = tasks.metric_isolation(user_intent_capture_agent, query)
    metric_discover_task = tasks.metric_discovery(discover_metric_info_agent)
    
    # print("Metric_isolator_task", metric_isolator_task)
    crew = Crew(
      agents=[
        user_intent_capture_agent,
        discover_metric_info_agent,
      ],
      tasks=[
        metric_isolator_task,
        metric_discover_task
      ],
      verbose=False,
      process=Process.sequential,
      step_callback=test_callback()
    )
    return crew
  

  def run(self, query: str):
    crew = self.crew_process(query)
    result = crew.kickoff()
    return result
  
  def call(
        self, goal: str, task: str, input_str: str, *args: Any, **kwargs: Any
    ) -> str:
    pass

In [24]:
query = "What is the most popular feature used by our paid subscribers"
# formatted_query = input(
#     dedent("""
#       {What is the most popular feature used by our paid subscribers}
#     """))
# print(formatted_query)
crew = MetricDiscoveryInputCrew()
result = crew.run(query)

Local variables: {}
Global variables: {'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'a ={"a" : MetricDiscoveryInputCrew()}', 'from crewai.process import Process\nfrom typing import Any\nclass MetricDiscoveryInputCrew(Expert):\n  name = "MetricDiscovery"\n  description = (\n       "Use this to answer the general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."\n    )\n  public_description = "Metric discovery crew"\n  arg_description = "Query related to metrics."\n\n  def __init__(self, query):\n    self.query = query\n    self.crew_process = self.crew_process()\n    # self.llm = llm_gpt\n  def crew_process(self):\n    agents = MetricDiscoveryAgent()\n    tasks = MetricDiscoveryTasks()\n    # 

In [27]:
print(type(result))
print(result)

<class 'str'>
Sorry, the query is out of scope.


In [61]:
experts_info = {"MetricDiscovery" : MetricDiscoveryInputCrew()}

In [9]:
from langchain import PromptTemplate
# Define the prompt template
prompt_create_objectives = PromptTemplate(
    input_variables=["goal", "chat_history"],
    template="""
You are an intelligent assistant that helps users break down their goals into high-level, smaller objectives. The system you are part of serves as an interface between users and a database containing various metrics. You have access to metadata of these metrics, including table columns information.
Goal: {goal}
{chat_history}
Using the provided goal and any relevant chat history, break down the goal into high-level, smaller objectives whose intents are different. Make sure each objective is clear and distinct.
) """)

goal ="Can you show me the average user signup rate for the past month, segmented by device type?"
chat_history = ""

prompt = prompt_create_objectives.format_prompt(
                goal=goal,
                chat_history=chat_history)


In [67]:
str(prompt)

"text='\\nYou are an intelligent assistant that helps users break down their goals into high-level, smaller objectives. The system you are part of serves as an interface between users and a database containing various metrics. You have access to metadata of these metrics, including table columns information.\\nGoal: Can you show me the average user signup rate for the past month, segmented by device type?\\n\\nUsing the provided goal and any relevant chat history, break down the goal into high-level, smaller objectives whose intents are different. Make sure each objective is clear and distinct.\\n) '"

In [68]:

response = llm_service.invoke(str(prompt))


In [72]:
response = response.content

In [73]:
response

"To achieve the goal of showing the average user signup rate for the past month, segmented by device type, we can break down the task into the following high-level, smaller objectives:\n\n1. **Data Retrieval Objective**: Access the database to retrieve user signup data for the past month. This involves:\n   - Identifying the correct database and table(s) that contain user signup information.\n   - Determining the relevant columns that store signup dates and device types.\n   - Constructing and executing a query to fetch the data within the date range of the past month.\n\n2. **Data Segmentation Objective**: Segment the retrieved data by device type. This requires:\n   - Grouping the signup data based on the device type column.\n   - Ensuring that each group represents a unique device type.\n\n3. **Calculation Objective**: Calculate the average signup rate for each device type. This involves:\n   - Counting the number of signups for each device type.\n   - Calculating the total number o

Objective breakdown into smaller task

In [29]:
from langchain import PromptTemplate
# Define the prompt template
prompt_template = PromptTemplate.from_template("""
You are an intelligent assistant whose only and only goal is to break down user's query {query} into high-level, smaller objectives in form of list. You serve as an interface between users and a database containing various metrics/KPIs/measures.
Strictly note that it is not necessary to break down the query in all the following intents. The intents are only for reference. If the query is straightforward and aligns with any one of the intent, then do not change the query. 
Try to minimize the number of objectives and keep them as high-level as possible. 
Each smaller objective should strictly fall into one of the following intent:
Discover Metrics:** If you'd like to measure something specific, I can identify relevant metrics. For example, are you interested in suggest some metrics based on objective?
Fetch Data:** If you already know the metric you're interested in, I can retrieve the data from the database.
Interpret Metrics:** Once you have the data, I can explain what it means in the context of your objective. 
Understand Metrics:** If you're unsure about a metric's definition or purpose, I can provide more information.

Don't add noise to the response.
Respond with a list.
However, if the query involves something else entirely, like action outside current intents, give empty list.""")
def complete(prompt, user_query):
  """
  Completes the prompt with the user objective.

  Args:
      prompt: The prompt template.
      user_objective: The user's objective.

  Returns:
      The completed prompt.
  """
  return prompt.format(query=user_query)

# Example usage
user_objective = "What are the top three metrics that indicate the overall health of our business?"
completed_prompt = complete(prompt_template, user_objective)

# Use the completed prompt with the AzureChatOpenAI model
response = llm_service.invoke(completed_prompt)

print(response)

content='- Discover Metrics: Identify the top three metrics that indicate the overall health of our business.' response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 283, 'total_tokens': 301}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None} id='run-c616fcc8-b9c9-48e5-870c-4a36a9eabe8f-0'


In [ ]:

prompt_template = """You are an intelligent assistant focused on helping users with metrics and data. Your primary goal is to break down user queries ({query}) into a concise list of high-level objectives. Intents: * Discover Metrics: Identify relevant metrics based on user objectives. (e.g., Suggest metrics to measure customer satisfaction) * Fetch Data: Retrieve data for a specific metric the user has already identified. * Analyze Trends: Analyze trends within the fetched data to provide deeper insights. * Interpret Metrics: Explain the meaning of the data in the context of the user's objective. * Understand Metrics: Provide information about a metric's definition and purpose. Important Notes: * Prioritize Clarity & Efficiency: Minimize the number of objectives and maintain a high-level overview. * Direct Match: If the query aligns perfectly with one intent (e.g., "What's the current customer churn rate?" for Fetch Data), don't alter it. * Outside Scope: If the query falls outside these intents (e.g., "Book a flight"), return an empty list. * Focus on User Need: Keep responses clear and concise, avoiding unnecessary information. * Output Format: Respond with a list containing the breakdown of objectives. **Here's the user query: {query} What is the high-level breakdown of objectives for this query? """

In [30]:
intents = {
    "discover_metrics": "Identify relevant metrics based on user objectives. (e.g., Suggest metrics to measure customer satisfaction)",
    "fetch_data": "Retrieve data for a specific metric the user has already identified.",
    # "analyze_trends": "Analyze trends within the fetched data to provide deeper insights.",
    "interpret_metrics": "Explain the meaning of the data in the context of the user's objective.",
    "understand_metrics": "Provide information about a metric's definition and purpose."
}
prompt_template = PromptTemplate.from_template("""You are an intelligent assistant focused on helping users with metrics and data. Your primary goal is to break down user queries ({query}) into a concise list of high-level objectives.

**Intents:**

{intents}

Important Notes:
* Alignment with Intents: The new objectives should strictly align with the provided intents.
* Prioritize Clarity & Efficiency:  Minimize the number of objectives and maintain a high-level overview. 
* Direct Match: If the query aligns perfectly with one intent (e.g., "What's the current customer churn rate?" for Fetch Data), don't alter it.
* Outside Scope: If the query falls outside these intents (e.g., "Book a flight"), return an empty list.
* Focus on User Need: Keep responses clear and concise, avoiding unnecessary information. 
* Output Format:  Respond with a list containing the breakdown of objectives.

**Here's the user query: {query}

What is the high-level breakdown of objectives for this query?
""")



# Format the intent descriptions for the prompt
formatted_intents = "\n".join([f"* {intent_name}: {description}" for intent_name, description in intents.items()])
query = "What dose the vessel name column represents?"
# Insert formatted intents into the prompt template
completed_prompt = prompt_template.format(intents=formatted_intents, query=query)
# Use the completed prompt with the AzureChatOpenAI model
response = llm_service.invoke(completed_prompt)

print(response)




content='- understand_metrics: Clarify what the "vessel name" column signifies in the dataset.' response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 288, 'total_tokens': 307}, 'model_name': 'gpt-4', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None} id='run-27e344c2-3c5b-4c84-8783-646da5ade68c-0'


In [35]:
import ast
import re
from typing import List, TypeVar
from langchain.schema import BaseOutputParser, OutputParserException
T = TypeVar("T")

class TaskOutputParser(BaseOutputParser[List[str]]):
    """
    Extension of LangChain's BaseOutputParser
    Responsible for parsing task creation output into a list of task strings
    """

    completed_tasks: List[str] = []

    def __init__(self, *, completed_tasks: List[str]):
        super().__init__()
        self.completed_tasks = completed_tasks

    def parse(self, text: str) -> List[str]:
        try:
            array_str = extract_array(text)
            all_tasks = [
                remove_prefix(task) for task in array_str if real_tasks_filter(task)
            ]
            return [task for task in all_tasks if task not in self.completed_tasks]
        except Exception as e:
            msg = f"Failed to parse tasks from completion '{text}'. Exception: {e}"
            raise OutputParserException(msg)

    def get_format_instructions(self) -> str:
        return """
        The response should be a JSON array of strings. Example:

        ["Search the web for NBA news", "Write some code to build a web scraper"]

        This should be parsable by json.loads()
        """

def handle_multiline_string(input_str: str) -> List[str]:
    # Handle multiline string as a list
    processed_lines = [
        re.sub(r".*?(\d+\..+)", r"\1", line).strip()
        for line in input_str.split("\n")
        if line.strip() != ""
    ]

    # Check if there is at least one line that starts with a digit and a period
    if any(re.match(r"\d+\..+", line) for line in processed_lines):
        return processed_lines
    else:
        # raise RuntimeError(f"Failed to extract array from {input_str}")
        print(f"Failed to extract array from {input_str}")
        return [input_str]

def extract_array(input_str: str) -> List[str]:
    regex = (
        r"\[\s*\]|"  # Empty array check
        r"(\[(?:\s*(?:\"(?:[^\"\\]|\\.)*\"|\'(?:[^\'\\]|\\.)*\')\s*,?)*\s*\])"
    )
    match = re.search(regex, input_str)
    if match is not None:
        return ast.literal_eval(match[0])
    else:
        return handle_multiline_string(input_str)
    
def remove_prefix(input_str: str) -> str:
    prefix_pattern = (
        r"^(Task\s*\d*\.\s*|Task\s*\d*[-:]?\s*|Step\s*\d*["
        r"-:]?\s*|Step\s*[-:]?\s*|\d+\.\s*|\d+\s*[-:]?\s*|^\.\s*|^\.*)"
    )
    return re.sub(prefix_pattern, "", input_str, flags=re.IGNORECASE)


def real_tasks_filter(input_str: str) -> bool:
    no_task_regex = (
        r"^No( (new|further|additional|extra|other))? tasks? (is )?("
        r"required|needed|added|created|inputted).*"
    )
    task_complete_regex = r"^Task (complete|completed|finished|done|over|success).*"
    do_nothing_regex = r"^(\s*|Do nothing(\s.*)?)$"

    return (
        not re.search(no_task_regex, input_str, re.IGNORECASE)
        and not re.search(task_complete_regex, input_str, re.IGNORECASE)
        and not re.search(do_nothing_regex, input_str, re.IGNORECASE)
    )

def parse_with_handling(parser: BaseOutputParser[T], completion: str) -> T:
    try:
        return parser.parse(completion)
    except OutputParserException as e:
        raise e
    

task_output_parser = TaskOutputParser(completed_tasks=[])


In [36]:
objectives = parse_with_handling(task_output_parser, response.content)
print(objectives)

Failed to extract array from - understand_metrics: Clarify what the "vessel name" column signifies in the dataset.
['- understand_metrics: Clarify what the "vessel name" column signifies in the dataset.']


In [37]:
#Available experts
from typing import Any, Dict, List, Optional, Union
from typing import Type, TypedDict

class ExpertDescription(TypedDict):
    """Representation of a callable expert"""
    name: str
    """The name of the expert."""
    description: str
    """A description of the expert."""
    parameters: dict[str, object]
    """The parameters of the expert."""

def get_expert_function(expert: Type[Expert]) -> ExpertDescription:
    """A function that will return the tool's function specification"""
    name = get_expert_name(expert)
    return {
        "name": name,
        "description": expert.description,
        "parameters": {
            "type": "object",
            "properties": {
                "reasoning": {
                    "type": "string",
                    "description": (
                        f"Reasoning is how the task will be accomplished with the current expert. "
                        "Detail your overall plan along with any concerns you have."
                        "Ensure this reasoning value is in the user defined langauge "
                    ),
                },
                "arg": {
                    "type": "string",
                    "description": expert.arg_description,
                },
            },
            "required": ["reasoning", "arg"],
        },
    }

def get_expert_name(expert: Type[Expert]) -> str:
    return expert.name

# def format_expert_name(expert_name: str) -> str:
#     return expert_name.lower()

def get_available_experts() -> List[Type[Expert]]:
    return [
        MetricDiscoveryInputCrew
    ]

In [38]:
available_experts = get_available_experts()
experts = list(map(get_expert_function, available_experts))
expert_descriptions = {expert['name'] : expert['description'] for expert in experts}

In [39]:
expert_descriptions

{'MetricDiscovery': 'Use this to answer the general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns.'}

In [40]:
experts

[{'name': 'MetricDiscovery',
  'description': 'Use this to answer the general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns.',
  'parameters': {'type': 'object',
   'properties': {'reasoning': {'type': 'string',
     'description': 'Reasoning is how the task will be accomplished with the current expert. Detail your overall plan along with any concerns you have.Ensure this reasoning value is in the user defined langauge '},
    'arg': {'type': 'string', 'description': 'Query related to metrics.'}},
   'required': ['reasoning', 'arg']}}]

In [51]:
analyze_task_prompt = PromptTemplate(
    template="""
    High level objective: "{goal}"
    Current task: "{task}"

    Based on this information, use the best expert to make progress or accomplish the task entirely. Each expert has a name and description of their expertise. 
    Select the correct expert by being smart and efficient. Ensure "reasoning" and only "reasoning" is in the english language.
    Following are the list of experts {experts}

    Note you MUST select a expert and return the name and reasoning of the expert in form of dictionary.
    """,
    input_variables=["goal", "task", "experts"],
)



In [65]:
import json 
for objective in objectives:
    analyze_prompt = analyze_task_prompt.format(goal=query, task=objective, experts=expert_descriptions)
    response = llm_service.invoke(analyze_prompt)
    selected_expert = json.loads(response.content)
    expert_object = experts_info.get(selected_expert['name'])
    output = expert_object.run(objective)

2024-06-05 14:11:50,170 - 140704419426240 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


Local variables: {}
Global variables: {'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'a ={"a" : MetricDiscoveryInputCrew()}', 'from crewai.process import Process\nfrom typing import Any\nclass MetricDiscoveryInputCrew(Expert):\n  name = "MetricDiscovery"\n  description = (\n       "Use this to answer the general user questions related to discovery, explaination, description, interpretation of metrics/measures/KPIs, tables or columns."\n    )\n  public_description = "Metric discovery crew"\n  arg_description = "Query related to metrics."\n\n  def __init__(self, query):\n    self.query = query\n    self.crew_process = self.crew_process()\n    # self.llm = llm_gpt\n  def crew_process(self):\n    agents = MetricDiscoveryAgent()\n    tasks = MetricDiscoveryTasks()\n    # 

In [66]:
print(output)

Sorry, the query is out of scope.
